# Boston Crime Database Creation

# Goal 

Goal of this project is to build a postgres database, with a schema, table user groups and users. Two groups will be created, one for data analysts, with read only capability and another for data scientists with readwrite capability. This project is part of the Dataquest Data Engineering with Python course track.


Import modules

In [1]:
import psycopg2
import csv
import os

### Step 1: Create a Database and Schema

A database named crimes_db will be created with a schema called crimes.

In [2]:
conn = psycopg2.connect(dbname = "dq",user = "dq")
cur = conn.cursor()

# Turn autocommit on to create the database
conn.autocommit = True

In [3]:
# Delete database in case I want to rerun all code
cur.execute("DROP DATABASE crimes_db")

# Create database
cur.execute("CREATE DATABASE crimes_db")

In [4]:
# Delete schema crimes in case I want to re-run all code
cur.execute("DROP SCHEMA crimes CASCADE")

# Create the Schema, crimes
cur.execute("CREATE SCHEMA crimes")

# Turn off autocommit
conn.autocommit = False

# Step 2: Create Tables

First, I will read in the crimes.csv file so I can a look at the headers and data. This information will be used to create the correct datatypes for the table.

In [5]:
with open("boston.csv") as file:
    reader = list(csv.reader(file))

# Save headers to its own variable
col_headers = reader[0]

# Assign first row of data to its own variable
first_row = reader[1]

print(col_headers,"\n",first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 
 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Now, to look at the datatypes for each column. To do this, I will create a function (get_col_value_set) that given a csv and column, will print the unique values in that column.

This function will be useful for two reasons:

1. Checking whether an enumerated datatype might be a good choice for representing a column.
2. Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

In [6]:
def get_col_value_set(csv_filename, col_index):
    # Create an empty set
    unique_values = set()
    
    with open(csv_filename,'r') as f:
        next(f)
        reader = csv.reader(f)

        for row in reader:
            unique_values.add(row[col_index])
    return unique_values

In [7]:
# Test the function
get_col_value_set("boston.csv",2)

{'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.',
 'A&B ON POLICE OFFICER',
 'ABDUCTION - INTICING',
 'AFFRAY',
 'AIRCRAFT INCIDENTS',
 'ANIMAL ABUSE',
 'ANIMAL CONTROL - DOG BITES - ETC.',
 'ANIMAL INCIDENTS',
 'ANNOYING AND ACCOSTIN',
 'ANNOYING AND ACCOSTING',
 'ARSON',
 'ASSAULT & BATTERY',
 'ASSAULT & BATTERY D/W - KNIFE',
 'ASSAULT & BATTERY D/W - OTHER',
 'ASSAULT & BATTERY D/W - OTHER ON POLICE OFFICER',
 'ASSAULT - AGGRAVATED',
 'ASSAULT - AGGRAVATED - BATTERY',
 'ASSAULT - SIMPLE',
 'ASSAULT D/W - KNIFE ON POLICE OFFICER',
 'ASSAULT D/W - OTHER',
 'ASSAULT SIMPLE - BATTERY',
 'AUTO THEFT',
 'AUTO THEFT - LEASED/RENTED VEHICLE',
 'AUTO THEFT - MOTORCYCLE / SCOOTER',
 'AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON',
 'AUTO THEFT - RECOVERED IN BY POLICE',
 'AUTO THEFT LEASE/RENT VEHICLE',
 'AUTO THEFT OTHER',
 'B&E NON-RESIDENCE DAY - FORCIBLE',
 'B&E NON-RESIDENCE DAY - NO FORCE',
 'B&E NON-RESIDENCE DAY - NO PROP TAKEN',
 'B&E NON-RESIDENCE NIGHT - FORCE',
 'B&E RESIDENCE DAY - N

Print the number of values in each column

In [8]:
for i in range(len(col_headers)):
    values = get_col_value_set("boston.csv",i)
    print(col_headers[i], len(values), sep = '\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


Generally, columns with low number of values are good candidates to be enumerated. Columns day_of_the_week only has 7 values, so that one is possible. 

Next, I will calculate the max length of each column. Knowing the length will help us save space in the database.

This is the maximum length for the description column:

In [9]:
descriptions = get_col_value_set("boston.csv", 2) 
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


Time to create the table!


In [10]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


The incident number is the unique key in this dataset. Two incidents cannot have the same number.

Offence code is a small integer number, with a max length of 4, so I chose smallint.

Decription is a pretty long field. The current max number of characters is 48, but the descriptions are pretty wordy, so I chose a length of 70 to accomdate for any new descriptions.

Date is pretty self explanatory. I saved the date field as a date.

Day of the week has a max length of 9, which is Wednesday.

Latitude can only have 11 characters so I stored it as a variable text field of 11 characters.

Longitude is also stored as a variable character field, but a length of 12.

In [11]:
cur.execute(
'''
    CREATE TABLE crimes.boston_crimes (
        incident_number TEXT PRIMARY KEY,
        offense_code smallint,
        description varchar(70),
        date date,
        day_of_the_week varchar(9),
        lat varchar(11),
        long varchar(12)
    )
'''
)


In [22]:
conn.commit()

## Load Data into the Boston Crimes Table

In [23]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

conn.commit()

In [26]:
# Verify it worked
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 1")
print(cur.fetchall())

[('1', 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', '42.35779134', '-71.13937053')]


# Create Groups and Users

As mentioned earlier, I want to create two groups: readonly and readwrite. Data analysts will use the readonly group while data scientists use the readwrite group.

### Create the two groups

In [27]:
# Ensure that no privileges were inherited from the public group and public schema.
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public")
conn.commit()

In [28]:
# Create the read only group and grant the select privilege only
cur.execute("CREATE GROUP readonly NOLOGIN")

# Grant the privileges
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")

cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly")

cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")

conn.commit()

In [29]:
# Create the readwrite group and grant the select, insert, delete, updates privileges
cur.execute("CREATE GROUP readwrite NOLOGIN")

# Grant the privileges
cur.execute("GRANT SELECT,DELETE,UPDATE, INSERT ON ALL TABLES IN SCHEMA crimes TO readwrite")

cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite")

cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")

conn.commit()

### Create two users

Create two users: data_analyst (readonly) and data_scient (readwrite)

In [30]:
# Create data_analyst
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")

# add data_analyst to readonly group
cur.execute("GRANT readonly TO data_analyst")

# Create data_scientist
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")

# Assign data_scientist to readwrite
cur.execute("GRANT readwrite TO data_scientist")

conn.commit()

Check that priviliges worked

In [31]:
cur.execute('''
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE grantee IN ('readonly', 'readwrite')
''')

print(cur.fetchall())

[('readonly', 'SELECT'), ('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]
